In [1]:
import os
os.getcwd()

'/mnt/ai-storage/personal/pradip/st-ultralytics/notebooks'

In [2]:
os.chdir("..")
os.getcwd()

'/mnt/ai-storage/personal/pradip/st-ultralytics'

In [9]:
import os.path as osp
import argparse
import cv2
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import Image
from loguru import logger

import torch
from ultralytics.utils.files import increment_path
from ultralytics import YOLO

from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.utils.yolov8 import download_yolov8s_model

In [10]:
# Raw Data for this task is kept at /mnt/ai-storage/project/adidas/data/raw_4k/AIC_2182_P2/
root_path = "/mnt/ai-storage/jira/imtec356"
os.path.exists(root_path)

True

In [11]:
selected_clip = 'ma25fad386_5100_5160'

In [12]:
video_path = os.path.join(root_path, 
                                selected_clip,
                               'pano.mp4')
video_path

'/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/pano.mp4'

In [13]:
court_mask_path = '/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/pano_mask.png'

In [18]:
cap = cv2.VideoCapture(video_path)
print(f"total frames from pano video: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}")

total frames from pano video: 1498.0


In [19]:
model_path = "scripts/models/20231231_yolov8x-albumentations.pt"

In [20]:
# download_yolov8s_model(model_path)

In [21]:
# Check source video
if not Path(video_path).is_file():
    raise FileNotFoundError(f"Source video at '{video_path}' not found.")

In [22]:
# Check model path
if not Path(model_path).is_file():
    # download_yolov8s_model(model_path)
    raise FileNotFoundError(f"Model path '{model_path}' is not a file.")

In [23]:
out_dir = None

In [24]:
# Output setup
if out_dir is None:
    out_dir = increment_path(Path(video_path).parent / "detection_results", exist_ok=True)
    out_dir.mkdir(parents=True, exist_ok=True)
out_dir

PosixPath('/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/detection_results')

In [25]:
str(out_dir)

'/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/detection_results'

In [27]:
str(Path(out_dir)/f"debug_frames/{1}.jpg")

'/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/detection_results/debug_frames/1.jpg'

In [29]:
out_path = None

In [30]:
if out_path is None:
    out_path = increment_path(Path(video_path).parent / "detection_results"/ (Path(video_path).stem+'.bbox'), exist_ok=True)
else:
    assert out_path.endswith(".bbox"), f"out_path must end with .bbox, got {out_path}"
out_path

PosixPath('/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/detection_results/pano.bbox')

In [32]:
str(out_path)+'.mp4'

'/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/detection_results/pano.bbox.mp4'

In [ ]:
# Check court mask path
use_court_mask = court_mask_path is not None
if use_court_mask:
    assert osp.exists(court_mask_path), f"Cannot find court mask at {court_mask_path}"
    logger.info(f"Use court mask: {court_mask_path}")
    court_mask = cv2.imread(court_mask_path)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(court_mask)

In [ ]:
model = YOLO(model_path)

In [ ]:
# Detect objects from classes 0 and 32 only
# classes = [0, 32]
# model.overrides["classes"] = classes

In [ ]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model=model,
    confidence_threshold=0.5,
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)

In [ ]:
# Output setup
save_dir = increment_path(Path(image_source).parent / "results_sahi" / "exp", True)
save_dir

In [ ]:
save_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
image_files = list(Path(image_source).rglob("*.[jp][pn]g"))

if not image_files:
    raise FileNotFoundError(f"No image files found in: {image_source}")

In [ ]:
player_detections = {}
ball_detections = {}

In [ ]:
img_path = image_files[0]
img_path

In [ ]:
results = get_sliced_prediction(
    str(img_path),
    detection_model,
    slice_height=512,
    slice_width=512,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
)

In [ ]:
results.export_visuals(export_dir="sahi_sample/",
                    text_size=5,
                    rect_th=None,
                    hide_labels=True,
                    hide_conf=True,
                    file_name="custom_yolov8_prediction_visual",)
# Image("sahi_sample/prediction_visual.png")

In [ ]:
visual = cv2.imread("sahi_sample/custom_yolov8_prediction_visual.png")
visual = cv2.cvtColor(visual, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,10))
plt.imshow(visual)

In [ ]:
frame_number = img_path.stem
frame_number

In [ ]:
# !pip install -U imantics

In [ ]:
player_frame_detections = {}
ball_frame_detections = {}

for bboxid, detection in enumerate(results.object_prediction_list):
    if detection.category.name == 'player':
        player_frame_detections[bboxid] = {
                                            'bbox': [int(v) for v in detection.bbox.to_xyxy()],
                                            'score': round(detection.score.value, 5)
                                          }
    else:
        ball_frame_detections[bboxid] = {
                                            'bbox': [int(v) for v in detection.bbox.to_xyxy()],
                                            'score': round(detection.score.value, 5)
                                        }

In [ ]:
player_detections[frame_number] = player_frame_detections
ball_detections[frame_number] = ball_frame_detections

In [ ]:
debug = True

In [ ]:
if debug:
    object_prediction_list = results.object_prediction_list
    boxes_list = []
    clss_list = []
    for ind, _ in enumerate(object_prediction_list):
        clss = object_prediction_list[ind].category.name
        boxes = (
            object_prediction_list[ind].bbox.minx,
            object_prediction_list[ind].bbox.miny,
            object_prediction_list[ind].bbox.maxx,
            object_prediction_list[ind].bbox.maxy,
        )

        boxes_list.append(boxes)
        clss_list.append(clss)

    frame = cv2.imread(str(img_path))

    # Create a copy of the original image to draw on
    frame_copy = frame.copy()

    for box, cls in zip(boxes_list, clss_list):
        x1, y1, x2, y2 = box
        cv2.rectangle(
            frame_copy, (int(x1), int(y1)), (int(x2), int(y2)), (56, 56, 255), 2
        )
        label = str(cls)
        t_size = cv2.getTextSize(label, 0, fontScale=0.6, thickness=1)[0]
        cv2.rectangle(
            frame_copy,
            (int(x1), int(y1) - t_size[1] - 3),
            (int(x1) + t_size[0], int(y1) + 3),
            (56, 56, 255) if label == "person" else (56, 255, 56),
            -1,
        )
        cv2.putText(
            frame_copy,
            label,
            (int(x1), int(y1) - 2),
            0,
            0.6,
            [255, 255, 255] if label == "person" else [0, 0, 0],
            thickness=1,
            lineType=cv2.LINE_AA,
        )

    frame_name = f"{frame_number}_dets.jpg"
    frame_path = save_dir / frame_name
    cv2.imwrite(
        str(frame_path),
        frame_copy,
    )

In [ ]:
frame_path

In [ ]:
output_dict = {
    "players": {},
    "ball": {},
    "debug": {
        "fps": fps,
        "image_h": height,
        "image_w": width,
        "model_name": args.model
    }
}


In [ ]:
detections = {}

for img_path in image_files:
    results = get_sliced_prediction(
        str(img_path),
        detection_model,
        slice_height=512,
        slice_width=512,
        overlap_height_ratio=0.2,
        overlap_width_ratio=0.2,
    )

    frame_number = int(img_path.stem)

    object_prediction_list = [
        res.to_coco_annotation().json for res in results.object_prediction_list
    ]

    detections[frame_number] = object_prediction_list

    if debug:
        object_prediction_list = results.object_prediction_list
        boxes_list = []
        clss_list = []
        for ind, _ in enumerate(object_prediction_list):
            clss = object_prediction_list[ind].category.name
            boxes = (
                object_prediction_list[ind].bbox.minx,
                object_prediction_list[ind].bbox.miny,
                object_prediction_list[ind].bbox.maxx,
                object_prediction_list[ind].bbox.maxy,
            )

            boxes_list.append(boxes)
            clss_list.append(clss)

        frame = cv2.imread(str(img_path))

        # Create a copy of the original image to draw on
        frame_copy = frame.copy()

        for box, cls in zip(boxes_list, clss_list):
            x1, y1, x2, y2 = box
            cv2.rectangle(
                frame_copy, (int(x1), int(y1)), (int(x2), int(y2)), (56, 56, 255), 2
            )
            label = str(cls)
            t_size = cv2.getTextSize(label, 0, fontScale=0.6, thickness=1)[0]
            cv2.rectangle(
                frame_copy,
                (int(x1), int(y1) - t_size[1] - 3),
                (int(x1) + t_size[0], int(y1) + 3),
                (56, 56, 255) if label == "person" else (56, 255, 56),
                -1,
            )
            cv2.putText(
                frame_copy,
                label,
                (int(x1), int(y1) - 2),
                0,
                0.6,
                [255, 255, 255] if label == "person" else [0, 0, 0],
                thickness=1,
                lineType=cv2.LINE_AA,
            )

        frame_name = f"{frame_number:05d}_dets.jpg"
        frame_path = save_dir / frame_name
        cv2.imwrite(
            str(frame_path),
            frame_copy,
        )

#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break

# cv2.destroyAllWindows()

In [ ]:
try:
    coco_out_path = f"{save_dir}/coco_results.pkl"
    print(f"Saving {coco_out_path}...")
    with open(coco_out_path, "wb") as f:
        pickle.dump(detections, f)
except Exception as e:
    print(e)
    print(f"Could not save {coco_out_path}")

print("Inference with SAHI is done.")


# Test function

In [3]:
import os
os.getcwd()

'/mnt/ai-storage/personal/pradip/st-ultralytics'

In [4]:
from scripts.inference_sahi import detect_bboxes_from_video

In [5]:
model_path = 'scripts/models/20231231_yolov8x-albumentations.pt'

In [6]:
video_path = '/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/pano.mp4'

In [7]:
court_mask_path = '/mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/pano_mask.png'

In [8]:
detect_bboxes_from_video(video_path = video_path,
                         model_path = model_path,
                         model_type = "yolov8",
                         start_time = 0,
                         end_time = 10,
                         court_mask_path = court_mask_path,
                         confidence_threshold= 0.5,
                         verbosity = 1,
                         debug=True)

2024-03-13 09:07:43.054 | INFO     | scripts.inference_sahi:detect_bboxes_from_video:160 - out_dir is: /mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/detection_results
2024-03-13 09:07:43.055 | INFO     | scripts.inference_sahi:detect_bboxes_from_video:166 - out_path is: /mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/detection_results/pano.bbox
2024-03-13 09:07:43.056 | INFO     | scripts.inference_sahi:detect_bboxes_from_video:172 - Use court mask: /mnt/ai-storage/jira/imtec356/ma25fad386_5100_5160/pano_mask.png
Player Detection for pano - 0  to 10: 0it [00:00, ?it/s]

verbosity 1 True
Performing prediction on 20 number of slices.


Player Detection for pano - 0  to 10: 0it [00:07, ?it/s]

Visualisation
Dumping final image to disk


TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'

In [28]:
out_path

NameError: name 'out_path' is not defined